In [224]:
import random
import igraph as ig
import numpy as np
from collections import Counter

In [106]:
G = ig.Graph.Barabasi(n=1_000, m=3)

In [107]:
G.vs.set_attribute_values("group","oridinary")

In [71]:
for node in G.vs:
    break

In [72]:
node.attributes()

{'group': 'oridinary'}

In [108]:
all_nodes = list(G.vs)

In [109]:
misinformed_nodes = np.random.choice(all_nodes, size = 500)

In [78]:
for node in misinformed_nodes:
    node['group'] = 'misinformed'

In [253]:
class MisinfoABM():
    """
    Class to implement ABM that considers a 'misinformed'
    subpopulation alongside an 'ordinary' subpopulation.
    Also, considers the role of homophily.
    """
    def __init__(self, n=1_000, m=3, prop_mis=.5, homophily=.5, prop_infect=.01):
        self._n = n
        self._m = m
        self._prop_mis = prop_mis
        self._homophily = homophily
        self._prop_infect = prop_infect
        self._counts = dict()
    
    def generate_barabasi(self):
        """
        Generate Barabasi-Albert network with self._n nodes, 
        self._m edges.
        """
        self.net = ig.Graph.Barabasi(n=self._n, m=self._m)
    
    def create_subpopulations(self):
        """
        Create the 'ordinary' and 'misinformed' subpopulations with
        the proportion of nodes for each being (1 - self._prop_mis)
        and self.prop_mis, respectively.
        """
        # Give all nodes a 'group' attribute and set them all to ordinary
        self.net.vs.set_attribute_values("group", "ordinary")
        
        # Randomly select the nodes to set as misinformed
        all_nodes = list(self.net.vs)
        num_misinformed_nodes = int(self._n * self._prop_mis)
        misinformed_nodes = np.random.choice(all_nodes, size = num_misinformed_nodes)
        for node in misinformed_nodes:
            node['group'] = 'misinformed'

    def initialize_compartments(self):
        """
        Set all nodes compartments as 'susceptible' and then
        infect self._infect_prop proportion of the nodes.
        """
        self.net.vs.set_attribute_values("compartment", "S")
        
        # Randomly select nodes to infect
        all_nodes = list(self.net.vs)
        num_nodes_to_infect = int(self._n * self._prop_infect)
        infected_nodes = np.random.choice(all_nodes, size = num_nodes_to_infect)
        for node in infected_nodes:
            node['compartment'] = 'I'
    
    def count_compartments(self):
        """
        Count the number of nodes in each compartment.
        """
        compartment_counter = Counter()
        for node in self.net.vs:
            compartment_counter[node['compartment']] += 1

        compartment_counter['S'] += 0
        compartment_counter['I'] += 0
        compartment_counter['R'] += 0
        
        state_num = len(self._counts)
        self._counts[state_num] = compartment_counter
        
    def spreading(self, mis_beta=0.16, ord_beta=0.08):
        recovery_days = 7
        gamma = 1/recovery_days
        
        all_nodes = list(self.net.vs)
        random.shuffle(all_nodes)

        for node in all_nodes:

            if node['compartment'] == "I":

                for neighb in node.neighbors():

                    if node['group'] == 'misinformed':
                        if random.random() <= mis_beta:
                            neighb['compartment'] = 'I'

                    elif node['group'] == 'ordinary':
                        if random.random() <= ord_beta:
                            neighb['compartment'] = 'I'
            
            elif node['compartment'] == "":
                if random.random() <= gamma:
                    neighb['compartment'] = 'I'
            
        self.count_compartments()

In [254]:
ABM = MisinfoABM()

In [255]:
ABM.generate_barabasi()

In [256]:
ABM.create_subpopulations()

In [257]:
ABM.initialize_compartments()

In [258]:
ABM.count_compartments()

In [259]:
ABM._counts

{0: Counter({'S': 990, 'I': 10, 'R': 0})}

In [260]:
ABM.spreading()

In [261]:
ABM._counts

{0: Counter({'S': 990, 'I': 10, 'R': 0}),
 1: Counter({'S': 984, 'I': 16, 'R': 0})}

In [232]:
count = 0

for node in ABM.net.vs:
    if node['compartment'] == 'I':
        count += 1

In [233]:
count

10

In [185]:
random.random()

0.2632494023655947

In [211]:
recovery_days = 7
gamma = 1/recovery_days

mis_beta = 0.16
ord_beta = 0.08

In [212]:
gamma

0.14285714285714285

In [214]:
ord_beta/gamma

0.56

In [222]:


compartment_counter = Counter()

for node in ABM.net.vs:
    
    compartment_counter[node['compartment']] += 1

compartment_counter['S'] += 0
compartment_counter['I'] += 0
compartment_counter['R'] += 0 

In [223]:
compartment_counter

Counter({'S': 990, 'I': 10, 'R': 0})

In [204]:
recovery_days = 7
gamma = 1/recovery_days

mis_beta = 0.16
ord_beta = 0.08


for node in ABM.net.vs:
    
    if node['compartment'] == "I":
        
        for neighb in node.neighbors():
            
            if node['group'] == 'misinformed':
                if random.random() <= mis_beta:
                    neighb['compartment'] = 'I'

            elif node['group'] == 'ordinary':
                if random.random() <= ord_beta:
                    neighb['compartment'] = 'I'

13
265
608
759
807
182
236


In [198]:
ABM.net.vs[377]

igraph.Vertex(<igraph.Graph object at 0x7fd8d21e5550>, 377, {'group': 'ordinary', 'compartment': 'I'})

In [174]:
if node['group'] == 'ordinary':
    print("not")

In [175]:
node['group'] == ''

'oridinary'

In [173]:
node

igraph.Vertex(<igraph.Graph object at 0x7fd8d21e5850>, 999, {'group': 'oridinary', 'compartment': 'S'})

In [166]:
len(list(ABM.net.vs))

1000

In [156]:
len(list(ABM.net.vs))

1000

In [152]:
ABM.net.vs[0]

igraph.Vertex(<igraph.Graph object at 0x7fd8d21e5750>, 0, {'group': 'oridinary', 'compartment': 'S'})

In [134]:
100000 * .01

1000.0

In [131]:
ABM.net.vs.set_attribute_values("compartment", "S")

In [132]:
ABM.net.vs[0]

igraph.Vertex(<igraph.Graph object at 0x7fd8d21e5450>, 0, {'group': 'misinformed', 'compart': 'S'})

In [ ]:
{""}